In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
#import dict_builder as db
import re
import pandas as pd
import requests
import numpy as np
#from __future__ import unicode_literals 
#from bosonnlp import BosonNLP
#nlp = BosonNLP('Yy7Tpb83.27824.Zdrfgp6JtKTf')
import os
import random
import pickle
import nltk
Porter = nltk.stem.PorterStemmer()

In [12]:
from bert_serving.client import BertClient
from scipy import spatial
bc = BertClient()

In [5]:
def readfile(filename,method='utf-8'):
    rawlist = []
    with open(filename,'r+', encoding=method) as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            rawlist.append(row)
    return rawlist

def writefile(filename,inputlist,method='utf-8'):
    with open(filename,'a+', encoding=method) as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for row in inputlist:
            writer.writerow(row)

In [6]:
def regex(a):
    reg = []
    #a = re.sub('[a-zA-Z]','',a)
    a = re.sub('[,、：；！？，。－"\']','#',a)
    a = re.sub('[「」(){}:;!?]','',a)
    a = re.sub('\n','',a)
    #a = re.sub('[(),{}\u3000\*\|=-\[\]\n.]','',a)
    #a = re.sub(u"\\（.*?\\）|\\『.*?』|\\「.*?」|\\〔.*?〕|\\[.*?]|\\〈.*?〉", '', a)
    reg = a.split('#')
    return reg

def en_regex(a):
    reg = []
    #a = re.sub('[a-zA-Z]','',a)
    a = re.sub('[{}.:;!?：；！？，。－"\']','#',a)
    a = re.sub('[、「」()]','',a)
    a = re.sub(',',' ',a)
    a = re.sub('\n','',a)
    #a = re.sub('[(),{}\u3000\*\|=-\[\]\n.]','',a)
    #a = re.sub(u"\\（.*?\\）|\\『.*?』|\\「.*?」|\\〔.*?〕|\\[.*?]|\\〈.*?〉", '', a)
    reg = a.split('#')
    return reg

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

# 從這裡開始跑

In [8]:
chcut = []
with (open('chcut.pickle', "rb")) as openfile:
    chcut = pickle.load(openfile)
encut = []
with (open('encut.pickle', "rb")) as openfile:
    encut = pickle.load(openfile)

# 用不到

ch_dic = {}
for i in range(0,len(chcut)):
    for j in range(0,len(chcut[i])):
        for word in chcut[i][j]:
            if ch_dic.get(word) == None:
                ch_dic[word] = [i]
            else:
                if i not in ch_dic[word]: 
                    ch_dic[word].append(i)

tmp = []
encut = []
for i in range(0,len(alignment)):
    for sent in alignment[i][1]:
        tmp.append(sent.split())
    encut.append(tmp)
    tmp = []

en_dic = {}
for i in range(0,len(encut)):
    for j in range(0,len(encut[i])):
        for word in encut[i][j]:
            #word = Porter.stem(word)
            if en_dic.get(word) == None:
                en_dic[word] = [i]
            else:
                if i not in en_dic[word]: 
                    en_dic[word].append(i)

bertvectors = {}
for i in range(0,len(encut)):
    if i%50==0:
        print(i)
    for j in range(0,len(encut[i])):
        for word in encut[i][j]:
            #word = Porter.stem(word)
            #print(word)
            if not word in bertvectors:
                if not word == '':
                    bertvectors[word] = bc1.encode([word])
                    
for i in range(0,len(chcut)):
    if i%50==0:
        print(i)
    for j in range(0,len(chcut[i])):
        for word in chcut[i][j]:
            try:
                tmp = adict[word]
            except:
                tmp = [word]
            for meaning in tmp:
                if meaning not in bertvectors:
                    if not meaning == '':
                        bertvectors[meaning] = bc.encode([meaning])

#計算有幾個match的文章
def co_occur(list1,list2):
    return len(list(set(list1).intersection(list2)))

ch_dic_freq = []
for key in ch_dic:
    ch_dic_freq.append(len(ch_dic[key]))
ch_dic_freq_mean = np.mean(ch_dic_freq)
ch_dic_freq_sd = np.std(ch_dic_freq)

def ch_weight(list1):
    list2 = []
    rlist = []
    for word in list1:
        list2.append([len(ch_dic[word]),word])
    list2 = sorted(list2,reverse=True)
    #print(list1)
    for i in range(0,len(list1)):
        for j in range(0,len(list2)):
            if list1[i] == list2[j][1]:
                rlist.append(j+1)
                #print(list1[i],j+1)
                break
    return(rlist)

def s2s_calculator(cs,es,chdic=ch_dic,endic=en_dic):
    s2s_rate = 0
    w2s_rate = 0
    rank = ch_weight(cs)
    weightsum = sum(rank)
    
    for i in range(0,len(cs)):    
        for en_word in es:
            numerator = co_occur(chdic[cs[i]],endic[Porter.stem(en_word)])
            denominator = len(chdic[cs[i]])+len(endic[Porter.stem(en_word)])-numerator
            w2w_rate = float(numerator/denominator)
            if w2w_rate>w2s_rate:
                #w2s_rate += float(w2w_rate/len(es))
                w2s_rate = w2w_rate
        s2s_rate += float(w2s_rate*(rank[i]/weightsum))
        #print(cs[i],rank[i]/weightsum,w2s_rate,s2s_rate)
        w2s_rate = 0
        w2w_rate = 0
    return(s2s_rate)

def another_s2s_calculator(cs,es,chdic=ch_dic,endic=en_dic):
    s2s_rate = 0
    w2s_rate = 0
    counter = 0
    for i in range(0,len(cs)):
        if not len(chdic[cs[i]])>(ch_dic_freq_mean+5*ch_dic_freq_sd):
            for en_word in es:
                numerator = co_occur(chdic[cs[i]],endic[Porter.stem(en_word)])
                denominator = len(chdic[cs[i]])+len(endic[Porter.stem(en_word)])-numerator
                w2w_rate = float(numerator/denominator)
                if w2w_rate>w2s_rate:
                    #w2s_rate += float(w2w_rate/len(es))
                    w2s_rate = w2w_rate
            counter = counter+1
            s2s_rate += float(w2s_rate)
            #print(cs[i],counter,w2s_rate,(s2s_rate/counter))
            w2s_rate = 0
            w2w_rate = 0
    if not counter==0:
        return(s2s_rate/counter)
    else:
        return 0

# 開始用的到

In [135]:
with open('bertvectors1.pickle', 'rb') as file:
    tmpd1 = pickle.load(file)
with open('bertvectors2.pickle', 'rb') as file:
    tmpd2 = pickle.load(file)
with open('bertvectors3.pickle', 'rb') as file:
    tmpd3 = pickle.load(file)
with open('bertvectors4.pickle', 'rb') as file:
    tmpd4 = pickle.load(file)
bertvectors = {**tmpd1, **tmpd2, **tmpd3, **tmpd4}
len(bertvectors)

38148

In [136]:
with open('chinese_bertvectors.pickle', 'rb') as file:
    chinese_bertvectors = pickle.load(file)
len(chinese_bertvectors)

17209

In [137]:
adictsrc = readfile('zh_en_dict.csv')
adict = dict()
for i in range(0,len(adictsrc)):
    adict[adictsrc[i][0]] = adictsrc[i][1:]
len(adict)

114442

In [138]:
with open('trans_matrix.pickle', 'rb') as file:
    transmatrix = pickle.load(file)
np.shape(transmatrix)

(768, 768)

# bert 相似度演算法

In [15]:
def bert_calculator(cs,es):
    s2s_rate = 0
    for i in range(0,len(cs)):
        try:
            translations = adict[cs[i]]
        except:
            translations = [cs[i]]
        tmpcossim = 0
        #print('*',translations)
        for trans in translations:
            #print(trans)
            if not trans == '':
                try:
                    bert_ch = bertvectors[trans]
                except:
                    bert_ch = bc.encode([trans])
                for en_word in es:
                    try:
                        bert_en = bertvectors[en_word]
                    except:
                        bert_en = bc.encode([en_word])
                    teststat = 1-spatial.distance.cosine(bert_en,bert_ch)
                    #print(trans,'/',en_word,teststat)
                    if (teststat > tmpcossim):
                        tmpcossim = teststat
        s2s_rate = s2s_rate + tmpcossim
    if(len(cs))==0:
        return 0
    s2s_rate = float(s2s_rate/len(cs))
    return(s2s_rate)

# bert 相似度演算法（利用轉置矩陣）

In [56]:
def bert_matrix(cs,es):
    s2s_rate = 0
    for i in range(0,len(cs)):
        ch_word = cs[i]
        tmpcossim = 0
        if not ch_word == '':
            try:
                bert_ch = chinese_bertvectors[ch_word]
            except:
                bert_ch = bc.encode([ch_word])
            bert_ch = np.dot(bert_ch,transmatrix)
            for en_word in es:
                try:
                    bert_en = bertvectors[en_word]
                except:
                    bert_en = bc.encode([en_word])
                teststat = 1-spatial.distance.cosine(bert_en,bert_ch)
                #print(teststat)
                if (teststat > tmpcossim):
                    tmpcossim = teststat
        s2s_rate = s2s_rate + tmpcossim
    if(len(cs))==0:
        return 0
    s2s_rate = float(s2s_rate/len(cs))
    return(s2s_rate)

In [89]:
def choose_your_algo(c,e,method='bert'):
    rate = 0
    if method=='rank':
        rate = s2s_calculator(c,e)
    elif method=='average':
        rate = another_s2s_calculator(c,e,chdic,endic)
    elif method=='bert':
        rate = bert_calculator(c,e)
    elif method=='matrix':
        rate = bert_matrix(c,e)    
    return rate

In [96]:
def word_output_combination(c,e,i,k):
    ctmp = ''
    etmp = ''
    for word in c:
        ctmp = ctmp + word
    for word in e:
        etmp = etmp+' '+word
    comber = str(i)
    for ktop in range(1,k):
        comber = comber +'+'+ str(i+ktop)
    return ctmp,etmp,comber

In [106]:
def reset_variables(i,k,j):
    igate = i+k
    jgate = j+1
    ceiling = 0
    rate = 0
    return igate,jgate,ceiling,rate

In [122]:
def greedy_operations(ceiling,tmp,en,i,k,j):
    ctmp,etmp,combstr = word_output_combination(tmp,en,i,k)
    if len(combstr)>1:
        print('Combination of {0}'.format(combstr))
    print('R:{0:4f}\nC:{1}\nE:{2}\n'.format(ceiling,ctmp,etmp))
    igate,jgate,ceiling,rate = reset_variables(i,k,j)
    return ctmp,etmp,igate,jgate,ceiling,rate

# greedy linear 演算法

In [129]:
def greedy(chlist,enlist,method='bert',threshold=0.85):
    igate = 0
    jgate = 0
    ceiling = 0
    rate = 0
    output = []
    ctmp = ''
    etmp = ''
    for i in range(0,len(chlist)):
        tmp = ''
        if i>=igate:
            for j in range(jgate,i+5):
                if j<len(enlist) and j>=0:
                    rate = choose_your_algo(chlist[i],enlist[j],method)    
                    ceiling = rate
                    #print(i,j,rate)
                    if rate>threshold:
                        tmp = chlist[i]
                        #print(tmp)
                        for k in range(1,5):
                            if (i+k)<len(chlist):
                                rate = choose_your_algo((tmp+chlist[i+k]),enlist[j],method)
                               
                                if rate > ceiling:
                                    tmp = tmp + chlist[i+k]
                                    ceiling = rate
                                    #print(tmp)
                                    if k==4:
                                    #Window用完
                                        ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(ceiling,
                                                                                           tmp,enlist[j],i,k,j)
                                        output.append([ctmp,etmp])
                                        break
                                else:
                                #結果沒變好直接吐出值
                                    ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(ceiling,
                                                                                           tmp,enlist[j],i,k,j)
                                    output.append([ctmp,etmp])
                                    break
                            else:
                            #中文用完
                                ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(ceiling,
                                                                                           tmp,enlist[j],i,k,j)
                                output.append([ctmp,etmp])
                                break
                        break
    return(output)

In [130]:
ch_template = [['本','論文','主要','討論','機器','學習'],['機器人','在','現代'],['對','人類','很','重要'],['同時','電腦','也','很','重要'],['我','喜歡','論文']]
en_template = [['This','paper','mainly','discuss','machine','learning'],['Robots','are','important','to','people','nowadays','and','so','do','computers'],['I','like','paper']]

In [131]:
for i in range(0,len(ch_template)): 
    for j in range(0,len(en_template)):
        print(i,j,bert_matrix(ch_template[i],en_template[j]))

0 0 0.7630445443170895
0 1 0.780333444938179
0 2 0.7790630640172993
1 0 0.7429939604586964
1 1 0.763077076752844
1 2 0.7583879150335374
2 0 0.7566632295554431
2 1 0.7762826827317164
2 2 0.7740580276648825
3 0 0.7537466386469027
3 1 0.774404109432125
3 2 0.7752456838927191
4 0 0.7724206424658083
4 1 0.8008964808373603
4 2 0.7916654306340094


In [139]:
greedy(ch_template,en_template)

R:0.903871
C:本論文主要討論機器學習
E: This paper mainly discuss machine learning

R:0.880259
C:對人類很重要
E: Robots are important to people nowadays and so do computers

R:0.885927
C:我喜歡論文
E: I like paper



[['本論文主要討論機器學習', ' This paper mainly discuss machine learning'],
 ['對人類很重要', ' Robots are important to people nowadays and so do computers'],
 ['我喜歡論文', ' I like paper']]

In [142]:
for i in range(10,15):
    print('-----Comparing set {0} using bert-----'.format(i))
    greedy(chcut[i],encut[i],method='matrix',threshold=0.83)

-----Comparing set 10 using bert-----
R:0.840216
C:含計數量詞的二階邏輯C2有著許多應用
E: It is well known that the satisfiability problem for C2 is decidable in nondeterministic exponential time NEXPTIME and the complexity is optimal

R:0.838386
C:一個著名的結果是
E: However the known techniques are quite complicated and they typically involve guessing a structure or a representation that satisfies the input formula which can be hard to implement

-----Comparing set 11 using bert-----
Combination of 3+4
R:0.839113
C:並使用文字來表示網路服務和進行比對因此文字比對的準確性會對服務比對的表現造成很大的影響
E: In text-based service matchmaking approach since the web service is treated as a plain text and use term tokens as the internal representation to match services the accuracy of the text comparison will affect the performance of service matchmaking

R:0.843955
C:2.從參考資料中取得文字關係
E: An experiment is also conducted based on an OWLS-TC V4 service matchmaking benchmark with hypothesis testing to compare our proposed approach with the iSeM approach

-----Comp

In [105]:
for i in range(10,15):
    print('-----Comparing set {0} using bert-----'.format(i))
    greedy(chcut[i],encut[i],method='bert')

-----Comparing set 10 using bert-----
Combination of 1+2
R:0.893147
C:含計數量詞的二階邏輯C2有著許多應用特別像是本體知識語言的應用
E: Two variable logic with counting quantifiers C2 has found many applications especially in ontology language such as OWL used in semantic web

R:0.879203
C:C2的可滿足性問題可以在非確定性指數時間NEXPTIME內決定
E: It is well known that the satisfiability problem for C2 is decidable in nondeterministic exponential time NEXPTIME and the complexity is optimal

R:0.884200
C:而且這樣的複雜度是最佳的
E: However the known techniques are quite complicated and they typically involve guessing a structure or a representation that satisfies the input formula which can be hard to implement

-----Comparing set 11 using bert-----
R:0.909035
C:在text-based的網路服務比對方法中
E: In text-based service matchmaking approach since the web service is treated as a plain text and use term tokens as the internal representation to match services the accuracy of the text comparison will affect the performance of service matchmaking

R:0.855585
C:因此文字比對的準

In [26]:
for i in range(10,15):
    print('-----Comparing set {0} using bert-----'.format(i))
    greedy(chcut[i],encut[i])

-----Comparing set 10 using bert-----
R:0.883479
C:含計數量詞的二階邏輯C2有著許多應用
E: Two variable logic with counting quantifiers C2 has found many applications especially in ontology language such as OWL used in semantic web

R:0.859864
C:特別像是本體知識語言的應用
E: It is well known that the satisfiability problem for C2 is decidable in nondeterministic exponential time NEXPTIME and the complexity is optimal

R:0.884200
C:而且這樣的複雜度是最佳的
E: However the known techniques are quite complicated and they typically involve guessing a structure or a representation that satisfies the input formula which can be hard to implement

-----Comparing set 11 using bert-----
R:0.909035
C:在text-based的網路服務比對方法中
E: In text-based service matchmaking approach since the web service is treated as a plain text and use term tokens as the internal representation to match services the accuracy of the text comparison will affect the performance of service matchmaking

R:0.855585
C:因此文字比對的準確性會對服務比對的表現造成很大的影響
E: An experiment is also conduc

In [15]:
evaluate_data = readfile('Bert_evaluation.csv')

In [16]:
c_eval_src = []
e_eval_src = []
#while not pin > len(evaluate_data):
for i in range(0,len(evaluate_data)):
    #length = random.randint(3,15)
    #if not pin+length>len(evaluate_data):
    c_eval_src.append(evaluate_data[i][2].split('#'))
    e_eval_src.append(evaluate_data[i][0].split())

In [17]:
pin = 0
tmp = []
tmp2 = []
c_eval = []
e_eval = []
for i in range(0,5):
    length = random.randint(5,15)
    pin = random.randint(0,len(evaluate_data))
    tmp = c_eval_src[pin:pin+length]
    tmp2 = e_eval_src[pin:pin+length]
    c_eval.append(tmp)
    e_eval.append(tmp2)

In [18]:
len(c_eval[0])

9

In [24]:
found=0
in_between=0
for i in range(0,1000):
    if i%100==0:
        print(i)
    score = bert_calculator(c_eval_src[i],e_eval_src[i+2000])
    if score>0.85:
        found+=1
    elif score>0.8:
        in_between+=1

0
100
200
300
400
500
600
700
800
900


In [25]:
print("Recall:",found/1000)
print("Recall(threshold=0.8):",(found+in_between)/1000)

Recall: 0.001
Recall(threshold=0.8): 0.151


In [157]:
for i in range(0,5):
    print('-----Comparing set {0} using bert-----'.format(i))
    greedy(c_eval[i],e_eval[i])

-----Comparing set 0 using bert-----
R:0.850348
C:隨著IIoT工業物聯網的發展，工廠的自動化系統變得更加開放，動態靈活，適應性強，複雜
E: With the development of IIoT Industrial IoT the autonomous system of the factory becomes more open dynamic flexible adaptable and complex

R:0.874579
C:這些系統的基本安全要求應該意識到負面影響的環境變化，並實施反饋閉環，不斷調整系統的行為
E: The fundamental safety requirement of those systems should be aware of the changes in the environment that affect negatively and be implemented the feedback closed loop that continuously adjust the behavior of systems

R:0.855538
C:在本論文中，為了處理安全運行時監控，我們使用Event-B規範和反應代理模型提出了安全規則的模型驅動方法。
E: In this thesis to deal with the safety runtime monitoring we proposed the model-driven approach of the safety rules by using the Event-B specification and reactive agent model

R:0.905850
C:這種方法使用Event-B的設計從指定受監控系統的安全規則轉變為事件B規範中的設計方法，以支持從安全規則到安全監控代理規則的中間轉換過程
E: This approach transforms from specifying the safety rule for the monitored system to the design method in Event-B specification using the design of Event-

# 沒用

c_eval_src = c_eval_src+chcut
e_eval_src = e_eval_src+encut

ch_eval_dic = {}
for i in range(0,len(c_eval_src)):
    for j in range(0,len(c_eval_src[i])):
        for word in c_eval_src[i][j]:
            if ch_eval_dic.get(word) == None:
                ch_eval_dic[word] = [i]
                #print(word)
            else:
                if i not in ch_eval_dic[word]: 
                    ch_eval_dic[word].append(i)

en_eval_dic = {}
for i in range(0,len(e_eval_src)):
    for j in range(0,len(e_eval_src[i])):
        for word in e_eval_src[i][j]:
            word = Porter.stem(word)
            if en_eval_dic.get(word) == None:
                en_eval_dic[word] = [i]
            else:
                if i not in en_eval_dic[word]: 
                    en_eval_dic[word].append(i)